## Import Library

In [1]:
import cv2 as cv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

## Import Dataset

In [2]:
train = '/kaggle/input/race-classification/train'
val = '/kaggle/input/race-classification/val'

In [3]:
train_labels = pd.read_csv('/kaggle/input/fairface-labels/fairface_label_train.csv')
val_labels = pd.read_csv('/kaggle/input/fairface-labels/fairface_label_val.csv')

## Overview 

In [4]:
train_labels

,file,age,gender,race,service_test
0,train/1.jpg,50-59,Male,East Asian,True
1,train/2.jpg,30-39,Female,Indian,False
2,train/3.jpg,3-9,Female,Black,False
3,train/4.jpg,20-29,Female,Indian,True
4,train/5.jpg,20-29,Female,Indian,True
...,...,...,...,...,...
86739,train/86740.jpg,20-29,Male,Indian,True
86740,train/86741.jpg,10-19,Male,Indian,True
86741,train/86742.jpg,more than 70,Female,Indian,True
86742,train/86743.jpg,10-19,Female,Black,True


## Preprocessing

## Modelling

## Evaluation